In [13]:
import re
import json
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [17]:
f = open('sample.json')
urls = json.load(f) 
restuarant_name = []
restuarant_logo = []
cuisine_tags = []
latitude = []
longitude = []
menu_items = []

for url in urls:
    response = requests.get(url)
    ## storing contents of the page
    html_data = response.content
    ##Beautiful Soup Object
    page = BeautifulSoup(html_data,'html.parser')
        
    ## restaurant_name
    
    restaurant_name_data = page.find('h1',{'class' : 'f-24 f-m-15 media-heading f-400 mb-0'}) ## searching using class_name attribute
    restuarant_name.append(restaurant_name_data.contents[0])
        
    ## restaurant_logo
    
    restaurant_logo_div = page.find('div',{'class': 'rest-logo-bg mr-4 mr-m-2 b-a'}) 
    restuarant_logo.append(restaurant_logo_div.find('img').attrs['src'])
        
    ##cuisine_tags
   
    cuising_tags_div = page.find('div',{'class' : 'col-sm-12 col-16 p-r-0'})
    cuisine_tags.append(cuising_tags_div.find('p').contents)
        
    ##getting json text

    data_json = page.find('script',id = '__NEXT_DATA__')
    json_data = json.loads(data_json.text)
        
    #latitude
    
    latitude_var = float(json_data['props']['pageProps']['gtmEventData']['restaurant']['latitude'])
    latitude.append(latitude_var)
        
    #longitude
    
    longitude_var = float(json_data['props']['pageProps']['gtmEventData']['restaurant']['longitude'])
    longitude.append(longitude_var)
        
    ## menu_items

    items = json_data['props']['pageProps']['initialMenuState']['menuData']['items']

    ##creating a list of dictionaries for every item in the menu with the required item details
    menu_items_ind = []

    for item in items:
        dict = {
            'name': "",
            'price': 0,
            'description' : "",
            'image' : ""
        }

        for key in dict:
          dict[key] = item[key]

        menu_items_ind.append(dict)
            
    menu_items.append(menu_items_ind)

res = {
        'restuarant_name':restuarant_name,
        'restaurant_logo': restuarant_logo,
        'cuisine_tags':cuisine_tags, 
        'latitude':latitude,
        'longitude': longitude,
        'menu_items':menu_items
       }
       
table_df = pd.DataFrame(res)
table_df.to_csv('results.csv')


,restuarant_name,restaurant_logo,cuisine_tags,latitude,longitude,menu_items
0,Gino's Deli,https://images.deliveryhero.io/image/talabat/r...,"[Sandwiches, Pasta, Italian]",25.064858,55.138377,"[{'name': 'Gino's Meal for One', 'price': 49, ..."
1,Pasta Della Nonna,https://images.deliveryhero.io/image/talabat/r...,"[Italian, Pasta, Salad]",25.065133,55.138278,"[{'name': 'Nonna's Meal for 1', 'price': 65, '..."
2,Pizzaro,https://images.deliveryhero.io/image/talabat/r...,"[Italian, Pizza, Pasta]",25.083309,55.146559,"[{'name': 'Mixed Seafood Pizza BOGO', 'price':..."
3,The Pasta Guyz,https://images.deliveryhero.io/image/talabat/r...,"[Italian, Pasta]",25.073776,55.132283,"[{'name': 'Cheesy Garlic Bread (Vegetarian)', ..."
4,Pizza Di Rocco,https://images.deliveryhero.io/image/talabat/r...,"[Pizza, Pasta, Italian]",25.065092,55.138791,"[{'name': 'Meal for One', 'price': 65, 'descri..."
5,Applebee's,https://images.deliveryhero.io/image/talabat/r...,"[Burgers, Fried Chicken, Salad]",25.101867,55.167853,"[{'name': 'Ribs 3-bone portion', 'price': 159..."
6,Rock House Sliders,https://images.deliveryhero.io/image/talabat/r...,"[Burgers, Sliders, American]",25.101892,55.167842,"[{'name': 'Side Chicks BOGO', 'price': 16, 'de..."
7,Subway,https://images.deliveryhero.io/image/talabat/r...,"[Healthy, Sandwiches, Breakfast]",25.108467,55.149043,"[{'name': 'Spicy Italian 6-Inch', 'price': 16,..."
8,Zizou Express Restaurant,https://images.deliveryhero.io/image/talabat/r...,"[Egyptian, Shawarma, Sandwiches]",25.115926,55.203443,"[{'name': 'Crebawy Offer', 'price': 105, 'desc..."
9,BRAVO AVOCADO,https://images.deliveryhero.io/image/talabat/r...,"[Healthy, Vegetarian, Salad]",25.118861,55.396361,"[{'name': 'AVOCADO TOAST + COFFEE', 'price': 2..."
